In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input,Flatten,GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt


In [ ]:
%%writefile content.json
{"intents":
[
  {
    "tags":"greeting",
    "inputs":["hello","hi there","nice to meet you","hi,is this a private organazation"],
    "responses":[
      "hi,welcome to the world hghest organazation",
      "hey you, do you need help me",
      "Hello, Hi there! How can I help you today?",
      "good bye"
    ],
    "tags":"workk",
    "inputs":["Hi, I need help creating a workout plan. Can you assist me?","I want to lose weight and build muscle, Okay, we can create a plan that includes both cardio and strength training. How many days a week do you want to work out?", "Yes, please, What time and date would you like to go?","Hi, I'm planning a trip to Paris. What can you recommend?","I would like to visit some museums and historical landmarks, The Louvre and the Eiffel Tower are must-see attractions. Do you need help with tickets or reservations?","Thank you, You're welcome! Let me know if there's anything else I can help you with."],
    "responses":[
      "Sure, I can help with that. What are your fitness goals?",
      "I want to work out five days a week, Great. Let's create a schedule that works for you.",
      "Tomorrow at 7 pm, Great, I've made a reservation for two at La Trattoria tomorrow at 7 pm. Enjoy your meal!",
      "Paris is a beautiful city with many attractions. What kind of activities are you interested in?",
      "Yes, please, I can help you with that. What date and time would you like to visit the Louvre?"
    ]
  }
]

}

Overwriting content.json


In [ ]:
with open("content.json") as content:
  data1 = json.load(content)

In [ ]:
tags = []
inputs = []
responses = {}
for intent in data1['intents']:
  responses[intent['tags']] = intent['responses']
  for lines in intent['inputs']:
    inputs.append(lines)
    tags.append(intent['tags'])
    

In [ ]:
data = pd.DataFrame({"inputs":inputs,"tags":tags})
data

,inputs,tags
0,"Hi, I need help creating a workout plan. Can y...",workk
1,"I want to lose weight and build muscle, Okay, ...",workk
2,"Yes, please, What time and date would you like...",workk
3,"Hi, I'm planning a trip to Paris. What can you...",workk
4,I would like to visit some museums and histori...,workk
5,"Thank you, You're welcome! Let me know if ther...",workk


In [ ]:
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd:''.join(wrd))
data

,inputs,tags
0,hi i need help creating a workout plan can you...,workk
1,i want to lose weight and build muscle okay we...,workk
2,yes please what time and date would you like t...,workk
3,hi im planning a trip to paris what can you re...,workk
4,i would like to visit some museums and histori...,workk
5,thank you youre welcome let me know if theres ...,workk


In [ ]:
toknizer = Tokenizer(num_words=200)
toknizer.fit_on_texts(data['inputs'])
train = toknizer.texts_to_sequences(data['inputs'])
x_train = pad_sequences(train)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])
y_train


array([0, 0, 0, 0, 0, 0])

In [ ]:
input_shape = x_train.shape[1]
print(input_shape)

32


In [ ]:
vocabulary = len(toknizer.word_index)
print("number of unique words is", vocabulary)
output_length = le.classes_.shape[0]
print("Output length", output_length)


number of unique words is 73
Output length 1


In [ ]:
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1, 10)(i)
x = LSTM(10, return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model = Model(i,x)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
train = model.fit(x_train, y_train,epochs=200)

Epoch 1/200
1/1 [==============================] - 4s 4s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 23ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 22ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 4/200
1/1 [==============================] - 0s 23ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 5/200
1/1 [==============================] - 0s 25ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 6/200
1/1 [==============================] - 0s 27ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 7/200
1/1 [==============================] - 0s 21ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 8/200
1/1 [==============================] - 0s 25ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 9/200
1/1 [==============================] - 0s 23ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 10/200
1/1 [==============================

In [ ]:
import random
while True:
  texts_p = []
  prediction_input = input('You: ')
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  prediction_input = toknizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)

  output = model.predict(prediction_input)
  output = output.argmax()

  response_tag = le.inverse_transform([output])[0]
  print("ASMAMAW CHATBOT:", random.choice(responses[response_tag]))




You: Hi, I need help creating a workout plan. Can you assist me?
1/1 [==============================] - 1s 756ms/step
ASMAMAW CHATBOT: Sure, I can help with that. What are your fitness goals?
You: hello
1/1 [==============================] - 0s 28ms/step
ASMAMAW CHATBOT: Sure, I can help with that. What are your fitness goals?
